# Train/Test/Validation Split

In [11]:
from dsm import datasets
from dsm.dsm_api import DeepSurvivalMachines
import numpy as np
from sklearn.model_selection import ParameterGrid
from sksurv.metrics import concordance_index_ipcw, brier_score,cumulative_dynamic_auc

### Loading the Support Dataset 

In [27]:
x, t, e = datasets.load_dataset('SUPPORT')

### Computing the required quantile of the data 

In [28]:
times = np.quantile(t[e == 1], [0.25, 0.5, 0.75]).tolist()

### Defining the train, test and validation size

In [29]:
train_size = round(9105 * 0.70)
val_size = round(9105 * 0.10)
test_size = round(9105 * 0.20)

### Splitting the data into train, test and validation sets

In [15]:
x_train = x[: train_size]
t_train = t[: train_size]
e_train = e[: train_size]

x_test = x[-test_size :]
t_test = t[-test_size :]
e_test = e[-test_size :]

x_val = x[train_size : train_size + val_size]
t_val = t[train_size : train_size + val_size]
e_val = e[train_size : train_size + val_size]

In [16]:
et_train = np.array([(e_train[i], t_train[i]) for i in range(len(e_train))],
                 dtype = [('e', bool), ('t', int)])
et_test = np.array([(e_test[i], t_test[i]) for i in range(len(e_test))],
                 dtype = [('e', bool), ('t', int)])
et_val = np.array([(e_val[i], t_val[i]) for i in range(len(e_val))],
                 dtype = [('e', bool), ('t', int)])

In [17]:
print(et_train.shape, et_test.shape, et_val.shape)

(6374,) (1821,) (910,)


### Defining the parameter grid

In [26]:
param_grid = {'k' : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
              'distribution' : ['LogNormal', 'Weibull']}
grids = ParameterGrid(param_grid)

### Validation 

In [19]:
for grid in grids:
    model = DeepSurvivalMachines(k = grid['k'], distribution = grid['distribution'])
    model.fit(x_train, t_train, e_train)
    out_risk = model.predict_risk(x_val, times)
    out_survival = model.predict_survival(x_val, times)

    cis = []
    brs = []
    for i, _ in enumerate(times):
        cis.append(concordance_index_ipcw(et_train, et_val, out_risk[:, i], times[i])[0])

    brs.append(brier_score(et_train, et_val, out_survival, times)[1])
    print("Concordance Index:", np.mean(cis))
    print("Brier Score:", np.mean(brs))
    cdauc = []
    for i, _ in enumerate(times):
        cdauc.append(cumulative_dynamic_auc(et_train, et_val, out_risk[:, i], times[i])[0])
    print("Cumulative_dynamic AUC ", np.mean(cdauc))
    print(grid)
    #scores  = scores + [[grid, np.mean(cis), np.mean(brs), np.mean(cdauc)]]

  0%|          | 0/10000 [00:00<?, ?it/s]/home/kishan/Documents/Projects/deepsurvival/deep_survival/lib/python3.8/site-packages/torch/autograd/__init__.py:130: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  Variable._execution_engine.run_backward(
  1%|          | 59/10000 [00:00<00:16, 585.51it/s]

Concordance Index: 0.560607191724184
Brier Score: 0.2021620748498746
Cumulative_dynamic AUC  0.5750584452945019
{'distribution': 'LogNormal', 'k': 1}


  1%|          | 59/10000 [00:00<00:16, 589.67it/s]

Concordance Index: 0.5821705308796168
Brier Score: 0.2009241698319314
Cumulative_dynamic AUC  0.5854638786373166
{'distribution': 'LogNormal', 'k': 2}


  0%|          | 35/10000 [00:00<00:28, 348.73it/s]

...

Concordance Index: 0.5534216773892559
Brier Score: 0.20349522562214395
Cumulative_dynamic AUC  0.5694979875996248
{'distribution': 'Weibull', 'k': 8}


  1%|          | 67/10000 [00:00<00:14, 666.44it/s]

Concordance Index: 0.509918346008642
Brier Score: 0.20704238283775456
Cumulative_dynamic AUC  0.5136175482948849
{'distribution': 'Weibull', 'k': 9}


100%|██████████| 1/1 [00:00<00:00,  3.90it/s]

Concordance Index: 0.5457867524346355
Brier Score: 0.20434785349912266
Cumulative_dynamic AUC  0.5518614097391491
{'distribution': 'Weibull', 'k': 10}


### Training  

In [23]:
model = DeepSurvivalMachines(k = 6, distribution = 'LogNormal')
model.fit(x_train, t_train, e_train)
out_risk = model.predict_risk(x_test, times)
out_survival = model.predict_survival(x_test, times)

100%|██████████| 1/1 [00:00<00:00,  4.83it/s]


### Prediction 

In [24]:
out_risk = model.predict_risk(x_test, times)
out_survival = model.predict_survival(x_test, times)

### Evaluation

In [25]:
cis = []
brs = []
for i, _ in enumerate(times):
    cis.append(concordance_index_ipcw(et_train, et_test, out_risk[:, i], times[i])[0])

brs.append(brier_score(et_train, et_test, out_survival, times)[1])
print("Concordance Index:", np.mean(cis))
print("Brier Score:", np.mean(brs))
cdauc = []
for i, _ in enumerate(times):
    cdauc.append(cumulative_dynamic_auc(et_train, et_test, out_risk[:, i], times[i])[0])
print("Cumulative_dynamic AUC ", np.mean(cdauc))

Concordance Index: 0.6576888784160551
Brier Score: 0.18412562484999506
Cumulative_dynamic AUC  0.684763312097342
